## Data Preprocessing
This Jupyter notebook outlines the data preparation steps taken to analyze and identify fraudulent transactions in the `Fraud Analytics Dataset.xlsx`. Below is a summary of the key steps performed:

#### 1. Relevant Imports
- Imported necessary libraries: `pandas`, `numpy`, `matplotlib.pyplot`, and `seaborn`.
- Configured seaborn settings for better visualization.

#### 2. Loading the Data
- Loaded the dataset from the specified Excel file using `pd.read_excel()`.
- Configured pandas to display up to 50 columns for a comprehensive view.

#### 3. Data Cleaning and Feature Engineering
- **Filling Null Values**: Replaced all missing values with 0 to ensure completeness of the dataset.
- **Creating `difference_amount` Column**: Calculated the difference between the requested and settled amounts to identify overpayments and underpayments.
- **Rearranging Columns**: Placed the `difference_amount` column next to the `payee_requested_amount` and `payee_settlement_amount` columns for clarity.

#### 4. Creating Checkpoints
- Saved a copy of the dataset at various stages to ensure the ability to revert to previous states if necessary.

#### 5. Descriptive Statistics
- Generated summary statistics for both categorical and numerical columns to understand the distribution and basic characteristics of the data.

#### 6. Engineering Date and Time Features
- **Date and Time Conversion**: Converted `dt_txn_comp` to datetime format and extracted the month and hour values.
- **Segmenting the Day**: Created a function to categorize transaction times into segments of the day (e.g., Morning, Afternoon).

#### 7. Reducing Dimensionality
- Dropped irrelevant columns to simplify the dataset and reduce dimensionality, focusing only on the most relevant features.

#### 8. Creating Target Variable
- Defined a binary target variable (`targets`) to indicate fraudulent transactions (1) and non-fraudulent transactions (0).

#### 9. Saving Processed Data
- Saved a subset of the dataset containing only fraudulent transactions for detailed analysis.
- Prepared and saved the final dataset with target variables for use in predictive modeling.


### Relevant Imports:

In [1]:
import pandas as pd
import numpy as np

### Loading the data:

In [2]:
file_path = r'datasets\Fraud Analytics Dataset.xlsx'
data = pd.read_excel(file_path)

pd.set_option('display.max_columns', 50)
data.head()

,txn_id,dt_txn_comp,txn_comp_time,txn_type,txn_subtype,initiating_channel_id,txn_status,error_code,payer_psp,payee_psp,remitter_bank,beneficiary_bank,payer_handle,payer_app,payee_handle,payee_app,payee_requested_amount,payee_settlement_amount,payer_location,payer_city,payer_state,payee_location,payee_city,payee_state,payer_os_type,payee_os_type,beneficiary_mcc_code,remitter_mcc_code,custref_transaction_ref,cred_type,cred_subtype,payer_app_id,payee_app_id,initiation_mode,dt_time_txn_compl,time_of_day
0,436,2023-08-11,00:00:00,Refund,Product Refund,4,Successful,NaN,Amazon Pay,BharatPe (for UPI and QR code payments),Allahabad Bank,Karur Vysya Bank,SCB,Standard Chartered UPI,KOTAK,BHIM KOTAK Pay,54020,54020,151001,Bathinda,Punjab,431601,Nanded,Maharashtra,Others,iOS,7277,6051,Reference 324,Debit Card,Prepaid Debit Card,BHIM Union Bank UPI App,AmazonPay,18,2023-08-11,Night
1,665,2021-10-17,00:00:00,Payment,Peer-to-Peer (P2P),16,Successful,NaN,ICICI iMobile,Square (for international transactions),Madhya Bihar Gramin Bank,Kotak Mahindra Bank,WASBI,WhatsApp Pay,HDFCBANKJD,JustDial,37670,37670,208001,Kanpur,Uttar Pradesh,302001,Jaipur,Rajasthan,MacOS,Windows,7012,5966,Reference 550,Overdraft,Business Overdraft,ESAF,BHIM Union Bank UPI App,18,2021-10-17,Night
2,726,2019-11-16,00:00:00,Withdrawal,ATM Withdrawal,3,Successful,NaN,Axis Pay,PhonePe for Merchants,Karur Vysya Bank,United Bank of India,KMBL,Khalijeb,UTKARSHBANK,UTKARSHBANK,22984,22984,125001,Hisar,Haryana,151001,Bathinda,Punjab,Android,Windows,1799,5047,Reference 611,Auto Loan,Used Car Loan,ICICI iMobile,BHIM Indus Pay,16,2019-11-16,Night
3,842,2023-09-05,00:00:00,Transfer,Inter-bank Transfer,0,Successful,NaN,MobiKwik,ICICI Merchant Services,HDFC Bank,Corporation Bank,IDBI,BHIM PAyWIZ by IDBI Bank,WASBI,WhatsApp Pay,62038,62038,151001,Bathinda,Punjab,403401,Ponda,Goa,Android,Windows,8641,822,Reference 729,Overdraft,Personal Overdraft,BHIM UPI APP,JusPay Technologies,2,2023-09-05,Night
4,2202,2021-09-18,00:00:00,Fee,Account Maintenance Fee,1,Successful,NaN,MasterCard,BharatPe (for UPI and QR code payments),Union Bank of India,Bank of India,UNIONBANK,BHIM Union Bank UPI App,NSDL,NSDL,72624,72624,768001,Sambalpur,Odisha,431601,Nanded,Maharashtra,Others,Windows,4225,5978,Reference 2068,Personal Loan,Unsecured Personal Loan,PREPAIDICICI,Cred,1,2021-09-18,Night


### Engineer features and cleanse the dataset:

In [3]:
# Fill Null values with 0:
data.fillna(0, inplace= True)

In [4]:
# The difference column shows the difference in the requested and settled amounts.
# Negative difference indicates overpayment while a positive difference indicates underpayments.
data['difference_amount'] = data.payee_requested_amount - data.payee_settlement_amount

In [5]:
# Rearrange the columns to show difference after requested and settled amount columns.
data = data[['txn_id', 'dt_txn_comp', 'txn_comp_time', 'txn_type',
       'txn_subtype', 'initiating_channel_id', 'txn_status', 'error_code',
       'payer_psp', 'payee_psp', 'remitter_bank', 'beneficiary_bank',
       'payer_handle', 'payer_app', 'payee_handle', 'payee_app',
       'payee_requested_amount', 'payee_settlement_amount',  'difference_amount',
       'payer_location', 'payer_city', 'payer_state', 'payee_location',
       'payee_city', 'payee_state', 'payer_os_type', 'payee_os_type',
       'beneficiary_mcc_code', 'remitter_mcc_code',
       'custref_transaction_ref', 'cred_type', 'cred_subtype',
       'payer_app_id', 'payee_app_id', 'initiation_mode',
       'dt_time_txn_compl', 'time_of_day']]

In [6]:
data.head()

,txn_id,dt_txn_comp,txn_comp_time,txn_type,txn_subtype,initiating_channel_id,txn_status,error_code,payer_psp,payee_psp,remitter_bank,beneficiary_bank,payer_handle,payer_app,payee_handle,payee_app,payee_requested_amount,payee_settlement_amount,difference_amount,payer_location,payer_city,payer_state,payee_location,payee_city,payee_state,payer_os_type,payee_os_type,beneficiary_mcc_code,remitter_mcc_code,custref_transaction_ref,cred_type,cred_subtype,payer_app_id,payee_app_id,initiation_mode,dt_time_txn_compl,time_of_day
0,436,2023-08-11,00:00:00,Refund,Product Refund,4,Successful,0,Amazon Pay,BharatPe (for UPI and QR code payments),Allahabad Bank,Karur Vysya Bank,SCB,Standard Chartered UPI,KOTAK,BHIM KOTAK Pay,54020,54020,0,151001,Bathinda,Punjab,431601,Nanded,Maharashtra,Others,iOS,7277,6051,Reference 324,Debit Card,Prepaid Debit Card,BHIM Union Bank UPI App,AmazonPay,18,2023-08-11,Night
1,665,2021-10-17,00:00:00,Payment,Peer-to-Peer (P2P),16,Successful,0,ICICI iMobile,Square (for international transactions),Madhya Bihar Gramin Bank,Kotak Mahindra Bank,WASBI,WhatsApp Pay,HDFCBANKJD,JustDial,37670,37670,0,208001,Kanpur,Uttar Pradesh,302001,Jaipur,Rajasthan,MacOS,Windows,7012,5966,Reference 550,Overdraft,Business Overdraft,ESAF,BHIM Union Bank UPI App,18,2021-10-17,Night
2,726,2019-11-16,00:00:00,Withdrawal,ATM Withdrawal,3,Successful,0,Axis Pay,PhonePe for Merchants,Karur Vysya Bank,United Bank of India,KMBL,Khalijeb,UTKARSHBANK,UTKARSHBANK,22984,22984,0,125001,Hisar,Haryana,151001,Bathinda,Punjab,Android,Windows,1799,5047,Reference 611,Auto Loan,Used Car Loan,ICICI iMobile,BHIM Indus Pay,16,2019-11-16,Night
3,842,2023-09-05,00:00:00,Transfer,Inter-bank Transfer,0,Successful,0,MobiKwik,ICICI Merchant Services,HDFC Bank,Corporation Bank,IDBI,BHIM PAyWIZ by IDBI Bank,WASBI,WhatsApp Pay,62038,62038,0,151001,Bathinda,Punjab,403401,Ponda,Goa,Android,Windows,8641,822,Reference 729,Overdraft,Personal Overdraft,BHIM UPI APP,JusPay Technologies,2,2023-09-05,Night
4,2202,2021-09-18,00:00:00,Fee,Account Maintenance Fee,1,Successful,0,MasterCard,BharatPe (for UPI and QR code payments),Union Bank of India,Bank of India,UNIONBANK,BHIM Union Bank UPI App,NSDL,NSDL,72624,72624,0,768001,Sambalpur,Odisha,431601,Nanded,Maharashtra,Others,Windows,4225,5978,Reference 2068,Personal Loan,Unsecured Personal Loan,PREPAIDICICI,Cred,1,2021-09-18,Night


### Checkpoint - 1

In [7]:
df = data.copy()

### Description of data:

In [8]:
df.describe(include=['O'])

,txn_comp_time,txn_type,txn_subtype,txn_status,error_code,payer_psp,payee_psp,remitter_bank,beneficiary_bank,payer_handle,payer_app,payee_handle,payee_app,payer_city,payer_state,payee_city,payee_state,payer_os_type,payee_os_type,custref_transaction_ref,cred_type,cred_subtype,payer_app_id,payee_app_id,time_of_day
count,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671,55671
unique,360,7,16,8,32,15,15,59,59,104,81,104,81,68,22,68,22,5,5,18857,7,15,81,81,4
top,04:36:00,Payment,Service Refund,Successful,0,MasterCard,Google Pay for Business,Indian Bank,Indian Bank,OKAXIS,WhatsApp Pay,OKAXIS,Google Pay,Munger,Haryana,Munger,Haryana,MacOS,Windows,Reference 324,Credit Card,Secured Personal Loan,WhatsApp Pay,WhatsApp Pay,Night
freq,262,10806,3675,50768,50768,3905,3954,2018,1903,1181,2246,1131,2236,2175,4580,2083,4523,11299,11430,3,11131,3927,2267,1997,17148


In [9]:
df.describe()

,txn_id,dt_txn_comp,initiating_channel_id,payee_requested_amount,payee_settlement_amount,difference_amount,payer_location,payee_location,beneficiary_mcc_code,remitter_mcc_code,initiation_mode,dt_time_txn_compl
count,55671.000000,55671,55671.000000,55671.000000,55671.000000,55671.000000,55671.000000,55671.000000,55671.000000,55671.000000,55671.000000,55671
mean,27836.000000,2021-06-22 10:02:16.185805824,9.520127,50224.339279,52509.940579,-2285.601300,495368.550035,496878.895062,5732.878985,5749.375132,9.451851,2021-06-22 21:57:58.887751168
min,1.000000,2019-01-01 00:00:00,0.000000,14.000000,14.000000,-317340.000000,110001.000000,110001.000000,0.000000,0.000000,0.000000,2019-01-01 06:34:00
25%,13918.500000,2020-04-03 00:00:00,4.000000,24988.000000,25636.000000,0.000000,306401.000000,306401.000000,4816.000000,4899.000000,4.000000,2020-04-03 16:58:00
50%,27836.000000,2021-06-24 00:00:00,10.000000,50264.000000,51808.000000,0.000000,517001.000000,517001.000000,5815.000000,5815.000000,10.000000,2021-06-24 08:38:00
75%,41753.500000,2022-09-10 00:00:00,15.000000,75530.000000,77826.000000,0.000000,700124.000000,700124.000000,7339.000000,7339.000000,14.000000,2022-09-10 04:40:00
max,55671.000000,2023-12-08 00:00:00,22.000000,99990.000000,416716.000000,27976.000000,854001.000000,854001.000000,9405.000000,9405.000000,22.000000,2023-12-08 22:02:00
std,16070.977755,NaN,6.458861,29003.090558,32316.082141,11412.568862,235472.926674,235390.743628,1943.351274,1916.672515,6.423692,NaN


### engineer features relating to date time:

In [10]:
# Convert dt_txn_comp to pandas DateTime format:
df['dt_txn_comp'] = pd.to_datetime(df.dt_txn_comp)
df['txn_comp_time'] = pd.to_datetime(df['txn_comp_time'], format="%H:%M:%S")
# Extract year value from dt_txn_comp column:
df['year'] = df.dt_txn_comp.dt.year
# Extract month value from dt_txn_comp column:
df['month'] = df.dt_txn_comp.dt.month
# Extract hour of the day value from txn_comp_time:
df['hour'] = df.txn_comp_time.dt.hour
df['txn_comp_time'] = df['txn_comp_time'].dt.time

In [11]:
df = df[['txn_id', 'dt_txn_comp', 'year', 'month', 'txn_comp_time', 'hour', 'txn_type',
       'txn_subtype', 'initiating_channel_id', 'txn_status', 'error_code',
       'payer_psp', 'payee_psp', 'remitter_bank', 'beneficiary_bank',
       'payer_handle', 'payer_app', 'payee_handle', 'payee_app',
       'payee_requested_amount', 'payee_settlement_amount',
       'difference_amount', 'payer_location', 'payer_city', 'payer_state',
       'payee_location', 'payee_city', 'payee_state', 'payer_os_type',
       'payee_os_type', 'beneficiary_mcc_code', 'remitter_mcc_code',
       'custref_transaction_ref', 'cred_type', 'cred_subtype',
       'payer_app_id', 'payee_app_id', 'initiation_mode',
       'dt_time_txn_compl', 'time_of_day']]
df.head()

,txn_id,dt_txn_comp,year,month,txn_comp_time,hour,txn_type,txn_subtype,initiating_channel_id,txn_status,error_code,payer_psp,payee_psp,remitter_bank,beneficiary_bank,payer_handle,payer_app,payee_handle,payee_app,payee_requested_amount,payee_settlement_amount,difference_amount,payer_location,payer_city,payer_state,payee_location,payee_city,payee_state,payer_os_type,payee_os_type,beneficiary_mcc_code,remitter_mcc_code,custref_transaction_ref,cred_type,cred_subtype,payer_app_id,payee_app_id,initiation_mode,dt_time_txn_compl,time_of_day
0,436,2023-08-11,2023,8,00:00:00,0,Refund,Product Refund,4,Successful,0,Amazon Pay,BharatPe (for UPI and QR code payments),Allahabad Bank,Karur Vysya Bank,SCB,Standard Chartered UPI,KOTAK,BHIM KOTAK Pay,54020,54020,0,151001,Bathinda,Punjab,431601,Nanded,Maharashtra,Others,iOS,7277,6051,Reference 324,Debit Card,Prepaid Debit Card,BHIM Union Bank UPI App,AmazonPay,18,2023-08-11,Night
1,665,2021-10-17,2021,10,00:00:00,0,Payment,Peer-to-Peer (P2P),16,Successful,0,ICICI iMobile,Square (for international transactions),Madhya Bihar Gramin Bank,Kotak Mahindra Bank,WASBI,WhatsApp Pay,HDFCBANKJD,JustDial,37670,37670,0,208001,Kanpur,Uttar Pradesh,302001,Jaipur,Rajasthan,MacOS,Windows,7012,5966,Reference 550,Overdraft,Business Overdraft,ESAF,BHIM Union Bank UPI App,18,2021-10-17,Night
2,726,2019-11-16,2019,11,00:00:00,0,Withdrawal,ATM Withdrawal,3,Successful,0,Axis Pay,PhonePe for Merchants,Karur Vysya Bank,United Bank of India,KMBL,Khalijeb,UTKARSHBANK,UTKARSHBANK,22984,22984,0,125001,Hisar,Haryana,151001,Bathinda,Punjab,Android,Windows,1799,5047,Reference 611,Auto Loan,Used Car Loan,ICICI iMobile,BHIM Indus Pay,16,2019-11-16,Night
3,842,2023-09-05,2023,9,00:00:00,0,Transfer,Inter-bank Transfer,0,Successful,0,MobiKwik,ICICI Merchant Services,HDFC Bank,Corporation Bank,IDBI,BHIM PAyWIZ by IDBI Bank,WASBI,WhatsApp Pay,62038,62038,0,151001,Bathinda,Punjab,403401,Ponda,Goa,Android,Windows,8641,822,Reference 729,Overdraft,Personal Overdraft,BHIM UPI APP,JusPay Technologies,2,2023-09-05,Night
4,2202,2021-09-18,2021,9,00:00:00,0,Fee,Account Maintenance Fee,1,Successful,0,MasterCard,BharatPe (for UPI and QR code payments),Union Bank of India,Bank of India,UNIONBANK,BHIM Union Bank UPI App,NSDL,NSDL,72624,72624,0,768001,Sambalpur,Odisha,431601,Nanded,Maharashtra,Others,Windows,4225,5978,Reference 2068,Personal Loan,Unsecured Personal Loan,PREPAIDICICI,Cred,1,2021-09-18,Night


In [12]:
df.month.unique(), df.year.unique(), df.hour.unique()

(array([ 8, 10, 11,  9,  4,  2,  6,  5,  1,  3,  7, 12]),
 array([2023, 2021, 2019, 2020, 2022]),
 array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22]))

In [13]:
def segment_day(hour):
    if 0 <= hour < 3:
        return 'LateNight'
    elif 3 <= hour < 6:
        return 'EarlyMorning'
    elif 6 <= hour < 9:
        return 'Morning'
    elif 9 <= hour < 12:
        return 'LateMorning'
    elif 12 <= hour < 15:
        return 'Afternoon'
    elif 15 <= hour < 18:
        return 'LateAfternoon'
    elif 18 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

In [14]:
df['time_of_day'] = df['hour'].apply(segment_day)

In [15]:
df.head()

,txn_id,dt_txn_comp,year,month,txn_comp_time,hour,txn_type,txn_subtype,initiating_channel_id,txn_status,error_code,payer_psp,payee_psp,remitter_bank,beneficiary_bank,payer_handle,payer_app,payee_handle,payee_app,payee_requested_amount,payee_settlement_amount,difference_amount,payer_location,payer_city,payer_state,payee_location,payee_city,payee_state,payer_os_type,payee_os_type,beneficiary_mcc_code,remitter_mcc_code,custref_transaction_ref,cred_type,cred_subtype,payer_app_id,payee_app_id,initiation_mode,dt_time_txn_compl,time_of_day
0,436,2023-08-11,2023,8,00:00:00,0,Refund,Product Refund,4,Successful,0,Amazon Pay,BharatPe (for UPI and QR code payments),Allahabad Bank,Karur Vysya Bank,SCB,Standard Chartered UPI,KOTAK,BHIM KOTAK Pay,54020,54020,0,151001,Bathinda,Punjab,431601,Nanded,Maharashtra,Others,iOS,7277,6051,Reference 324,Debit Card,Prepaid Debit Card,BHIM Union Bank UPI App,AmazonPay,18,2023-08-11,LateNight
1,665,2021-10-17,2021,10,00:00:00,0,Payment,Peer-to-Peer (P2P),16,Successful,0,ICICI iMobile,Square (for international transactions),Madhya Bihar Gramin Bank,Kotak Mahindra Bank,WASBI,WhatsApp Pay,HDFCBANKJD,JustDial,37670,37670,0,208001,Kanpur,Uttar Pradesh,302001,Jaipur,Rajasthan,MacOS,Windows,7012,5966,Reference 550,Overdraft,Business Overdraft,ESAF,BHIM Union Bank UPI App,18,2021-10-17,LateNight
2,726,2019-11-16,2019,11,00:00:00,0,Withdrawal,ATM Withdrawal,3,Successful,0,Axis Pay,PhonePe for Merchants,Karur Vysya Bank,United Bank of India,KMBL,Khalijeb,UTKARSHBANK,UTKARSHBANK,22984,22984,0,125001,Hisar,Haryana,151001,Bathinda,Punjab,Android,Windows,1799,5047,Reference 611,Auto Loan,Used Car Loan,ICICI iMobile,BHIM Indus Pay,16,2019-11-16,LateNight
3,842,2023-09-05,2023,9,00:00:00,0,Transfer,Inter-bank Transfer,0,Successful,0,MobiKwik,ICICI Merchant Services,HDFC Bank,Corporation Bank,IDBI,BHIM PAyWIZ by IDBI Bank,WASBI,WhatsApp Pay,62038,62038,0,151001,Bathinda,Punjab,403401,Ponda,Goa,Android,Windows,8641,822,Reference 729,Overdraft,Personal Overdraft,BHIM UPI APP,JusPay Technologies,2,2023-09-05,LateNight
4,2202,2021-09-18,2021,9,00:00:00,0,Fee,Account Maintenance Fee,1,Successful,0,MasterCard,BharatPe (for UPI and QR code payments),Union Bank of India,Bank of India,UNIONBANK,BHIM Union Bank UPI App,NSDL,NSDL,72624,72624,0,768001,Sambalpur,Odisha,431601,Nanded,Maharashtra,Others,Windows,4225,5978,Reference 2068,Personal Loan,Unsecured Personal Loan,PREPAIDICICI,Cred,1,2021-09-18,LateNight


### Checkpoint 2:

In [16]:
fin_df = df.copy()

In [17]:
fin_df.columns.values

array(['txn_id', 'dt_txn_comp', 'year', 'month', 'txn_comp_time', 'hour',
       'txn_type', 'txn_subtype', 'initiating_channel_id', 'txn_status',
       'error_code', 'payer_psp', 'payee_psp', 'remitter_bank',
       'beneficiary_bank', 'payer_handle', 'payer_app', 'payee_handle',
       'payee_app', 'payee_requested_amount', 'payee_settlement_amount',
       'difference_amount', 'payer_location', 'payer_city', 'payer_state',
       'payee_location', 'payee_city', 'payee_state', 'payer_os_type',
       'payee_os_type', 'beneficiary_mcc_code', 'remitter_mcc_code',
       'custref_transaction_ref', 'cred_type', 'cred_subtype',
       'payer_app_id', 'payee_app_id', 'initiation_mode',
       'dt_time_txn_compl', 'time_of_day'], dtype=object)

In [18]:
# dropping irrelevant columns to reduce dimensionality:
columns_to_drop = ['txn_id', 'dt_txn_comp', 'txn_comp_time', 'initiating_channel_id',
                   'payer_psp', 'payee_psp', 'payer_location', 'payer_city',
                   'payee_location', 'payee_city', 'payer_os_type', 'payee_os_type',
                   'beneficiary_mcc_code', 'remitter_mcc_code', 'custref_transaction_ref',
                    'payer_app_id', 'payee_app_id', 'initiation_mode', 'dt_time_txn_compl']

columns_to_keep = [col for col in fin_df.columns if col not in columns_to_drop]

In [19]:
fin_df = fin_df[columns_to_keep]
fin_df.head()

,year,month,hour,txn_type,txn_subtype,txn_status,error_code,remitter_bank,beneficiary_bank,payer_handle,payer_app,payee_handle,payee_app,payee_requested_amount,payee_settlement_amount,difference_amount,payer_state,payee_state,cred_type,cred_subtype,time_of_day
0,2023,8,0,Refund,Product Refund,Successful,0,Allahabad Bank,Karur Vysya Bank,SCB,Standard Chartered UPI,KOTAK,BHIM KOTAK Pay,54020,54020,0,Punjab,Maharashtra,Debit Card,Prepaid Debit Card,LateNight
1,2021,10,0,Payment,Peer-to-Peer (P2P),Successful,0,Madhya Bihar Gramin Bank,Kotak Mahindra Bank,WASBI,WhatsApp Pay,HDFCBANKJD,JustDial,37670,37670,0,Uttar Pradesh,Rajasthan,Overdraft,Business Overdraft,LateNight
2,2019,11,0,Withdrawal,ATM Withdrawal,Successful,0,Karur Vysya Bank,United Bank of India,KMBL,Khalijeb,UTKARSHBANK,UTKARSHBANK,22984,22984,0,Haryana,Punjab,Auto Loan,Used Car Loan,LateNight
3,2023,9,0,Transfer,Inter-bank Transfer,Successful,0,HDFC Bank,Corporation Bank,IDBI,BHIM PAyWIZ by IDBI Bank,WASBI,WhatsApp Pay,62038,62038,0,Punjab,Goa,Overdraft,Personal Overdraft,LateNight
4,2021,9,0,Fee,Account Maintenance Fee,Successful,0,Union Bank of India,Bank of India,UNIONBANK,BHIM Union Bank UPI App,NSDL,NSDL,72624,72624,0,Odisha,Maharashtra,Personal Loan,Unsecured Personal Loan,LateNight


### Create a dataset with targets.
### If 'txn_subtype' == 'Fraudulent Transaction', target = 1 else 0

In [20]:
def get_target(txn_subtype):
    return 1 if txn_subtype == 'Fraudulent Transaction' else 0

fin_df['targets'] = fin_df['txn_subtype'].apply(get_target)
fin_df.head()

,year,month,hour,txn_type,txn_subtype,txn_status,error_code,remitter_bank,beneficiary_bank,payer_handle,payer_app,payee_handle,payee_app,payee_requested_amount,payee_settlement_amount,difference_amount,payer_state,payee_state,cred_type,cred_subtype,time_of_day,targets
0,2023,8,0,Refund,Product Refund,Successful,0,Allahabad Bank,Karur Vysya Bank,SCB,Standard Chartered UPI,KOTAK,BHIM KOTAK Pay,54020,54020,0,Punjab,Maharashtra,Debit Card,Prepaid Debit Card,LateNight,0
1,2021,10,0,Payment,Peer-to-Peer (P2P),Successful,0,Madhya Bihar Gramin Bank,Kotak Mahindra Bank,WASBI,WhatsApp Pay,HDFCBANKJD,JustDial,37670,37670,0,Uttar Pradesh,Rajasthan,Overdraft,Business Overdraft,LateNight,0
2,2019,11,0,Withdrawal,ATM Withdrawal,Successful,0,Karur Vysya Bank,United Bank of India,KMBL,Khalijeb,UTKARSHBANK,UTKARSHBANK,22984,22984,0,Haryana,Punjab,Auto Loan,Used Car Loan,LateNight,0
3,2023,9,0,Transfer,Inter-bank Transfer,Successful,0,HDFC Bank,Corporation Bank,IDBI,BHIM PAyWIZ by IDBI Bank,WASBI,WhatsApp Pay,62038,62038,0,Punjab,Goa,Overdraft,Personal Overdraft,LateNight,0
4,2021,9,0,Fee,Account Maintenance Fee,Successful,0,Union Bank of India,Bank of India,UNIONBANK,BHIM Union Bank UPI App,NSDL,NSDL,72624,72624,0,Odisha,Maharashtra,Personal Loan,Unsecured Personal Loan,LateNight,0


### Create and save a copy of fraudulent data for furthur analysis:

In [21]:
fraudulent_data = fin_df[fin_df.targets == 1]
fraudulent_data = fraudulent_data.iloc[:, :-1]
fraudulent_data.drop(columns='txn_subtype', inplace= True)
fraudulent_data.to_csv(r"datasets\fraudulent_data.csv")

### Drop txn_subtype from fin_df and save for predictive modelling.

In [22]:
fin_df1 = fin_df.drop(columns=['txn_subtype'])
fin_df1.to_csv(r'datasets/financial_data_with_targets.csv')

## Conclusion
In this notebook, we successfully prepared a dataset for analyzing fraudulent transactions. The key steps included loading the data, cleaning it, engineering new features, and reducing dimensionality. By following these steps, we ensured that the dataset is well-structured and ready for further analysis.

### Summary of Achievements:
- **Data Loading and Inspection**: Loaded the dataset and performed initial inspections to understand its structure and content.
- **Data Cleaning**: Handled missing values by filling them with zeros to ensure completeness.
- **Feature Engineering**: Created new features such as `difference_amount`, extracted month and hour values from date and time columns, and segmented the day based on transaction times.
- **Dimensionality Reduction**: Dropped irrelevant columns to focus on the most important features.
- **Target Variable Creation**: Defined a binary target variable to distinguish between fraudulent and non-fraudulent transactions.
- **Data Saving**: Saved intermediate and final datasets for further analysis and predictive modeling.

### Next Steps:
With the data now prepared, the next steps involve:
- **Exploratory Data Analysis (EDA)**: Conduct a detailed EDA to uncover patterns and insights in the data.
- **Predictive Modeling**: Develop and evaluate machine learning models to predict fraudulent transactions.
- **Model Evaluation and Optimization**: Assess model performance using appropriate metrics and optimize for better accuracy.

The clean and well-structured dataset we have now will provide a solid foundation for these next steps, ultimately aiding in the detection and prevention of fraudulent transactions.


# END OF DOCUMENT